# Курсовая работа по курсу "Рекомендательные системы". <a class='anchor' id='begin'>

## Содержание
* [Итоговый проект по курсу "Машинное обучение в бизнесе"](#begin) 
    * [1. Задание](#task)
    * [2. Загрузка и просмотр данных](#load_help)
    * [3. Первый уровень. Поиск лучшего метода](#modeling_1)
    * [4. Второй уровень. Улучшение базовой модели 1-го уровня](#modeling_2)
    * [5. Оценка](#evaluation)

## 1. Задание <a class='anchor' id='task'>

* Закрепить навыки, полученные на курсе рекомендательные системы;

* Выявить закономерности(паттерны) поведения(покупок) покупателя на предложенных данных, сформировать рекомендации на их основании и предложить товары для некторого интервала времени покупок покупателя(проверка на retail_test1 -> 3 недели с 96-й по 98-ю включительно). Применяется только "тёплый старт (когда есть итория покупок покупателя).

* Преодолеть порог метрики precision@5 = 0.22 на retail_test1 без дробления retail_test1 на части(по неделям, дням, корзинам и т.д).

Пояснения:

Данная курсовая работа построена на базе домашних заданий, которые были выполнены в рамках данного курса.

## 2. Загрузка и просмотр данных <a class='anchor' id='load_help'>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Модель второго уровня
from lightgbm import LGBMClassifier
from sklearn.metrics import precision_score

import os, sys
module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)

# Написанные нами функции
from src.metrics import precision_at_k, recall_at_k
from src.utils import prefilter_items
from src.recommenders import MainRecommender

import warnings
warnings.filterwarnings('ignore')

In [2]:
def get_precision_at_k(data_with_recs_and_preds, k=5):
    train_top_5_rec = pd.DataFrame(columns = data_with_recs_and_preds.columns.tolist())
    for u_id in data_with_recs_and_preds['user_id'].unique():
        recs = data_with_recs_and_preds[data_with_recs_and_preds['user_id']==u_id].sort_values('pred', ascending=False).head(k)
        recs['pred'] = 1.0
        train_top_5_rec = train_top_5_rec.append(recs, ignore_index = True)
        
    result = precision_score(train_top_5_rec['target'], train_top_5_rec['pred'])
    return result

Загрузка и просмотр

In [3]:
data = pd.read_csv('data/retail_train.csv')
item_features = pd.read_csv('data/product.csv')
user_features = pd.read_csv('data/hh_demographic.csv')

In [4]:
data.head()

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.60,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.00,1631,1,0.0,0.0
2,2375,26984851472,1,1036325,1,0.99,364,-0.30,1631,1,0.0,0.0
3,2375,26984851472,1,1082185,1,1.21,364,0.00,1631,1,0.0,0.0
4,2375,26984851472,1,8160430,1,1.50,364,-0.39,1631,1,0.0,0.0


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2396804 entries, 0 to 2396803
Data columns (total 12 columns):
 #   Column             Dtype  
---  ------             -----  
 0   user_id            int64  
 1   basket_id          int64  
 2   day                int64  
 3   item_id            int64  
 4   quantity           int64  
 5   sales_value        float64
 6   store_id           int64  
 7   retail_disc        float64
 8   trans_time         int64  
 9   week_no            int64  
 10  coupon_disc        float64
 11  coupon_match_disc  float64
dtypes: float64(4), int64(8)
memory usage: 219.4 MB


In [6]:
item_features.head()

,PRODUCT_ID,MANUFACTURER,DEPARTMENT,BRAND,COMMODITY_DESC,SUB_COMMODITY_DESC,CURR_SIZE_OF_PRODUCT
0,25671,2,GROCERY,National,FRZN ICE,ICE - CRUSHED/CUBED,22 LB
1,26081,2,MISC. TRANS.,National,NO COMMODITY DESCRIPTION,NO SUBCOMMODITY DESCRIPTION,
2,26093,69,PASTRY,Private,BREAD,BREAD:ITALIAN/FRENCH,
3,26190,69,GROCERY,Private,FRUIT - SHELF STABLE,APPLE SAUCE,50 OZ
4,26355,69,GROCERY,Private,COOKIES/CONES,SPECIALTY COOKIES,14 OZ


In [7]:
item_features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 92353 entries, 0 to 92352
Data columns (total 7 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   PRODUCT_ID            92353 non-null  int64 
 1   MANUFACTURER          92353 non-null  int64 
 2   DEPARTMENT            92353 non-null  object
 3   BRAND                 92353 non-null  object
 4   COMMODITY_DESC        92353 non-null  object
 5   SUB_COMMODITY_DESC    92353 non-null  object
 6   CURR_SIZE_OF_PRODUCT  92353 non-null  object
dtypes: int64(2), object(5)
memory usage: 4.9+ MB


In [8]:
user_features.head()

,AGE_DESC,MARITAL_STATUS_CODE,INCOME_DESC,HOMEOWNER_DESC,HH_COMP_DESC,HOUSEHOLD_SIZE_DESC,KID_CATEGORY_DESC,household_key
0,65+,A,35-49K,Homeowner,2 Adults No Kids,2,None/Unknown,1
1,45-54,A,50-74K,Homeowner,2 Adults No Kids,2,None/Unknown,7
2,25-34,U,25-34K,Unknown,2 Adults Kids,3,1,8
3,25-34,U,75-99K,Homeowner,2 Adults Kids,4,2,13
4,45-54,B,50-74K,Homeowner,Single Female,1,None/Unknown,16


In [9]:
user_features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 801 entries, 0 to 800
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   AGE_DESC             801 non-null    object
 1   MARITAL_STATUS_CODE  801 non-null    object
 2   INCOME_DESC          801 non-null    object
 3   HOMEOWNER_DESC       801 non-null    object
 4   HH_COMP_DESC         801 non-null    object
 5   HOUSEHOLD_SIZE_DESC  801 non-null    object
 6   KID_CATEGORY_DESC    801 non-null    object
 7   household_key        801 non-null    int64 
dtypes: int64(1), object(7)
memory usage: 50.2+ KB


**Предобработка и формирование выборок**

In [10]:
# column processing
item_features.columns = [col.lower() for col in item_features.columns]
user_features.columns = [col.lower() for col in user_features.columns]

item_features.rename(columns={'product_id': 'item_id'}, inplace=True)
user_features.rename(columns={'household_key': 'user_id'}, inplace=True)


# Важна схема обучения и валидации!
# -- давние покупки -- | -- 6 недель -- | -- 3 недель -- 
# подобрать размер 2-ого датасета (6 недель) --> learning curve (зависимость метрики recall@k от размера датасета)
val_lvl_1_size_weeks = 6
val_lvl_2_size_weeks = 3

data_train_lvl_1 = data[data['week_no'] < data['week_no'].max() - (val_lvl_1_size_weeks + val_lvl_2_size_weeks)]

Сделаем отбор топ-5000 товаров, поскольу делать для всех товаров лично мне не позволяют ресурсы. Ждать обработку слишком долго, а работу с GPU нам не показали, а у меня самостоятельно не получилось.

Тут важно понимать, что некоторые пользователи, которые не покупали ТОП-5000 отвалятся из тренировочной выборки, но мы их рассматривать и не будем в дальнейшем.

In [11]:
n_items_before = data_train_lvl_1['item_id'].nunique()
n_users_before = data_train_lvl_1['user_id'].nunique()

data_train_lvl_1 = prefilter_items(data_train_lvl_1, item_features=item_features, take_n_popular=5000)

n_items_after = data_train_lvl_1['item_id'].nunique()
n_users_after = data_train_lvl_1['user_id'].nunique()
print('Decreased # items from {} to {}'.format(n_items_before, n_items_after))
print('Decreased # users from {} to {}'.format(n_users_before, n_users_after))
users_list = data_train_lvl_1['user_id'].unique()

Decreased # items from 83685 to 5001
Decreased # users from 2498 to 2497


In [12]:
# Valid for Model 1
data_val_lvl_1 = data[(data['week_no'] >= data['week_no'].max() - (val_lvl_1_size_weeks + val_lvl_2_size_weeks)) &
                      (data['week_no'] < data['week_no'].max() - (val_lvl_2_size_weeks))]

data_val_lvl_1 = data_val_lvl_1[data_val_lvl_1['user_id'].isin(users_list)]

In [13]:
# Train for Model 2
data_train_lvl_2 = data_val_lvl_1.copy()  # Для наглядности. Далее мы добавим изменения, и они будут отличаться

In [14]:
# Valid for Model 2
data_val_lvl_2 = data[data['week_no'] >= data['week_no'].max() - val_lvl_2_size_weeks]

## 3. Первый уровень. Поиск лучшего метода <a class='anchor' id='modeling_1'>
    
В процессе экспериментов ALS(n_factors=20, regularization=0.001, iterations=15) показал себя лучше всего.

In [15]:
recommender = MainRecommender(data_train_lvl_1)

  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/5001 [00:00<?, ?it/s]

In [16]:
result_lvl_1 = data_val_lvl_1.groupby('user_id')['item_id'].unique().reset_index()
result_lvl_1.columns=['user_id', 'actual']
result_lvl_1.head(2)

,user_id,actual
0,1,"[853529, 865456, 867607, 872137, 874905, 87524..."
1,2,"[15830248, 838136, 839656, 861272, 866211, 870..."


In [17]:
result_lvl_1['model_1'] = result_lvl_1['user_id'].apply(lambda x: recommender.get_als_recommendations(x, n=200))

In [18]:
recall_lvl_1=result_lvl_1.apply(lambda row: recall_at_k(row['model_1'], row['actual'], k=200), axis=1).mean()
recall_lvl_1

0.20452326190320885

In [19]:
result_lvl_1.head(2)

,user_id,actual,model_1
0,1,"[853529, 865456, 867607, 872137, 874905, 87524...","[862349, 1070820, 1058997, 1082185, 885290, 98..."
1,2,"[15830248, 838136, 839656, 861272, 866211, 870...","[1082185, 1106523, 1029743, 995242, 1133018, 5..."


**Формирирование тренировочного датасета для второй модели**

In [20]:
train_users_lvl_2 = pd.DataFrame(data_train_lvl_2['user_id'].unique())
train_users_lvl_2.columns = ['user_id']

# train_users = data_train_lvl_1['user_id'].unique()
train_users_lvl_2 = train_users_lvl_2[train_users_lvl_2['user_id'].isin(users_list)]

train_users_lvl_2['candidates'] = train_users_lvl_2['user_id'].apply(lambda x: recommender.get_als_recommendations(x, n=200))

In [21]:
train_users_lvl_2.head()

,user_id,candidates
0,2070,"[908531, 879755, 981760, 878996, 1127831, 9043..."
1,2021,"[1082185, 1133018, 1106523, 995242, 1029743, 9..."
2,1753,"[1085604, 1080414, 879755, 1124729, 1051069, 1..."
3,2120,"[1082185, 1029743, 995242, 981760, 1106523, 11..."
4,1346,"[1082185, 995242, 1029743, 981760, 1127831, 10..."


In [22]:
s = train_users_lvl_2.apply(lambda x: pd.Series(x['candidates']), axis=1).stack().reset_index(level=1, drop=True)
s.name = 'item_id'

train_users_lvl_2 = train_users_lvl_2.drop('candidates', axis=1).join(s)
train_users_lvl_2['flag'] = 1

In [23]:
train_users_lvl_2.head(2)

,user_id,item_id,flag
0,2070,908531,1
0,2070,879755,1


In [24]:
targets_lvl_2 = data_train_lvl_2[['user_id', 'item_id']].copy()
targets_lvl_2['target'] = 1  # тут только покупки 

targets_lvl_2 = train_users_lvl_2.merge(targets_lvl_2, on=['user_id', 'item_id'], how='left')

targets_lvl_2['target'].fillna(0, inplace=True)
targets_lvl_2.drop('flag', axis=1, inplace=True)

In [25]:
targets_lvl_2.head(5)

,user_id,item_id,target
0,2070,908531,1.0
1,2070,879755,0.0
2,2070,981760,1.0
3,2070,878996,0.0
4,2070,1127831,0.0


In [26]:
targets_lvl_2 = targets_lvl_2.merge(item_features, on='item_id', how='left')
targets_lvl_2 = targets_lvl_2.merge(user_features, on='user_id', how='left')

targets_lvl_2.head(2)

,user_id,item_id,target,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc
0,2070,908531,1.0,69,GROCERY,Private,FLUID MILK PRODUCTS,CHOCOLATE MILK,,45-54,U,50-74K,Unknown,Unknown,1,None/Unknown
1,2070,879755,0.0,103,GROCERY,National,SOFT DRINKS,SFT DRNK SNGL SRV BTL CARB (EX,20 OZ,45-54,U,50-74K,Unknown,Unknown,1,None/Unknown


## 4. Второй уровень. Улучшение базовой модели 1-го уровня <a class='anchor' id='modeling_2'>

Обучим вторую модель

In [27]:
X_train = targets_lvl_2.drop('target', axis=1)
y_train = targets_lvl_2[['target']]

In [28]:
cat_feats = X_train.columns[2:].tolist()
X_train[cat_feats] = X_train[cat_feats].astype('category')

cat_feats

['manufacturer',
 'department',
 'brand',
 'commodity_desc',
 'sub_commodity_desc',
 'curr_size_of_product',
 'age_desc',
 'marital_status_code',
 'income_desc',
 'homeowner_desc',
 'hh_comp_desc',
 'household_size_desc',
 'kid_category_desc']

In [29]:
lgb = LGBMClassifier(objective='binary', max_depth=7, categorical_column=cat_feats)
lgb.fit(X_train, y_train)

train_preds = lgb.predict_proba(X_train)

Оценим Precision на тренировочной выборке:

In [30]:
targets_lvl_2['pred'] = train_preds[:, 1]
train_result = targets_lvl_2[['user_id', 'item_id', 'target','pred']]
train_result.head()

,user_id,item_id,target,pred
0,2070,908531,1.0,0.181814
1,2070,879755,0.0,0.180333
2,2070,981760,1.0,0.310694
3,2070,878996,0.0,0.053340
4,2070,1127831,0.0,0.083568


In [31]:
precision_train = get_precision_at_k(train_result, k=5)
precision_train

0.3474907063197026

## 5. Оценка <a class='anchor' id='evaluation'>

Оценим работу двухуровневой модели на тестовой выборке : `data_val_lvl_2`

**Прогоняем тестовую выборку через модель №1**

In [32]:
test_users_lvl_2 = pd.DataFrame(data_train_lvl_2['user_id'].unique())
test_users_lvl_2.columns = ['user_id']

test_users_lvl_2 = test_users_lvl_2[test_users_lvl_2['user_id'].isin(users_list)]
test_users_lvl_2['candidates'] = test_users_lvl_2['user_id'].apply(lambda x: recommender.get_als_recommendations(x, n=200))

In [33]:
test_users_lvl_2.head()

,user_id,candidates
0,2070,"[908531, 879755, 981760, 878996, 1127831, 9043..."
1,2021,"[1082185, 1133018, 1106523, 995242, 1029743, 9..."
2,1753,"[1085604, 1080414, 879755, 1124729, 1051069, 1..."
3,2120,"[1082185, 1029743, 995242, 981760, 1106523, 11..."
4,1346,"[1082185, 995242, 1029743, 981760, 1127831, 10..."


In [34]:
s = test_users_lvl_2.apply(lambda x: pd.Series(x['candidates']), axis=1).stack().reset_index(level=1, drop=True)
s.name = 'item_id'

test_users_lvl_2 = test_users_lvl_2.drop('candidates', axis=1).join(s)
test_users_lvl_2['flag'] = 1

In [35]:
test_users_lvl_2.head()

,user_id,item_id,flag
0,2070,908531,1
0,2070,879755,1
0,2070,981760,1
0,2070,878996,1
0,2070,1127831,1


In [36]:
targets_test = data_val_lvl_2[['user_id', 'item_id']].copy()
targets_test['target'] = 1  # тут только покупки 

targets_test = test_users_lvl_2.merge(targets_test, on=['user_id', 'item_id'], how='left')

targets_test['target'].fillna(0, inplace=True)
targets_test.drop('flag', axis=1, inplace=True)

targets_test = targets_test.merge(item_features, on='item_id', how='left')
targets_test = targets_test.merge(user_features, on='user_id', how='left')

targets_test.head(2)

,user_id,item_id,target,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc
0,2070,908531,1.0,69,GROCERY,Private,FLUID MILK PRODUCTS,CHOCOLATE MILK,,45-54,U,50-74K,Unknown,Unknown,1,None/Unknown
1,2070,908531,1.0,69,GROCERY,Private,FLUID MILK PRODUCTS,CHOCOLATE MILK,,45-54,U,50-74K,Unknown,Unknown,1,None/Unknown


**Прогоняем валидационную выборку через модель №2**

In [37]:
X_test = targets_test.drop('target', axis=1)
y_test = targets_test[['target']]

In [38]:
cat_feats = X_test.columns[2:].tolist()
X_test[cat_feats] = X_test[cat_feats].astype('category')

cat_feats

['manufacturer',
 'department',
 'brand',
 'commodity_desc',
 'sub_commodity_desc',
 'curr_size_of_product',
 'age_desc',
 'marital_status_code',
 'income_desc',
 'homeowner_desc',
 'hh_comp_desc',
 'household_size_desc',
 'kid_category_desc']

In [39]:
test_preds = lgb.predict_proba(X_test)

In [40]:
targets_test['pred'] = test_preds[:, 1]
test_result = targets_test[['user_id', 'item_id', 'target','pred']]
test_result.head()

,user_id,item_id,target,pred
0,2070,908531,1.0,0.181814
1,2070,908531,1.0,0.181814
2,2070,908531,1.0,0.181814
3,2070,879755,0.0,0.180333
4,2070,981760,0.0,0.310694


In [41]:
precision_test = get_precision_at_k(test_result, k=5)
precision_test

0.25678438661710035

In [42]:
print(f"Precision@5 Train: {np.round(precision_train, 2)}")
print(f"Precision@5 Test: {np.round(precision_test, 2)}")
print("Precision@5 Threshold: 0.22")
print("")
print(f"Precision@5 Test > Precision@5 Threshold: {np.round(precision_test, 2) > 0.22}")

Precision@5 Train: 0.35
Precision@5 Test: 0.26
Precision@5 Threshold: 0.22

Precision@5 Test > Precision@5 Threshold: True


**Вывод:** Порог метрики precision@5 преодолен на тестовых данных (3 недели с 96-й по 98-ю включительно), курсовая работа выполнена.

$$Threshold: \;\;\; precision@5 = 0.22$$

$$Train: \;\;\; precision@5 = 0.35$$

$$Test: \;\;\; precision@5 = 0.26$$

---